# 測試多模態模型提取 PDF 中需要欄位：

### 1. 所得相關資料提取

- 目前使用 Vertex AI ( Gemini-1.5-flash 多模態模型 ) 來提取 PDF 中的資料 
- 明確定義我們需要的提取的內容
- 定義 JSON
- 設計 Prompt 並讓 Gemini 理解任務
- 去得輸出，並改成可解析的結構

In [21]:
import vertexai
from dotenv import load_dotenv
from vertexai.generative_models import GenerativeModel, Part

project_id = "nthurc-aisearch-202406"

load_dotenv()
vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-001")

json_format = '''{
  "taxYear": 提取此所得資料清單[年度]資訊，位於文檔頂部,
  "IssuingAuthority": 提取此所得稅[核發單位]文字資訊放於此欄位,
  "taxpayerName": 提取[所得人姓名],
  "taxpayerID": 提取所得人[統一編號]，位於文檔中部偏右，是所得人的唯一識別碼,
  "reportDate": 提取此報告[查調日期]，位於文檔右上角,
  "totalIncome": 提取此所得人總所得，位於文檔下方有一個[給付總額合計]的資訊,
}'''


prompt = f"""
You are now a professional PDF document extractor. I will provide you with a property income document from the Ministry of Finance of Taiwan. You will extract the information from each page and organize it into JSONL to return to me. I will clearly define the content I need, and you will extract the value and return it based on the content in the PDF.
```json format : {json_format}```
Please carefully review the content of each page of the PDF file and extract the information to generate the corresponding JSON response. The value can accept output in Traditional Chinese. Think carefully before outputting and then return the conclusion. Your answer is very important to my career.

JSONL :
"""

pdf_file_uri = "gs://nthurc-docai-poc/申請資料-4要件/所得/所得14.pdf"
pdf_file = Part.from_uri(pdf_file_uri, mime_type="application/pdf")
contents = [pdf_file, prompt]

response = model.generate_content(contents)
print(response.text)


```jsonl
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃德旺", "taxpayerID": "R124075657", "reportDate": "111年07月11日", "totalIncome": "724,894"}
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃春萬", "taxpayerID": "R120335830", "reportDate": "111年07月11日", "totalIncome": "107"}
{"taxYear": "110年度", "IssuingAuthority": "財政部臺北國稅局中正分局", "taxpayerName": "黃麗珍", "taxpayerID": "Q221609858", "reportDate": "111年07月11日", "totalIncome": "0"}
```


In [23]:
import json
import re

json_strings = re.sub(r'^[^\{]+', '', response.text).rstrip('`').strip().split('\n')
data = []
for json_string in json_strings:
    # 檢查是否為空字串，若是則跳過
    if not json_string.strip():  # 或 if json_string == '':
        continue

    try:
        data.append(json.loads(json_string))
    except json.JSONDecodeError as e:
        print(json_string)
        print(f"Error decoding JSON: {e}")

# 現在 `data` 變數包含了轉換後的 JSON 資料
print(data)

```
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
[{'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃德旺', 'taxpayerID': 'R124075657', 'reportDate': '111年07月11日', 'totalIncome': '724,894'}, {'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃春萬', 'taxpayerID': 'R120335830', 'reportDate': '111年07月11日', 'totalIncome': '107'}, {'taxYear': '110年度', 'IssuingAuthority': '財政部臺北國稅局中正分局', 'taxpayerName': '黃麗珍', 'taxpayerID': 'Q221609858', 'reportDate': '111年07月11日', 'totalIncome': '0'}]


## 2. 戶籍謄本相關資料提取
- 跟上一個方法一樣，通過定義需要的欄位後,讓多模態模型進行視覺的資料提取並轉換成文字回傳